In [1]:
import pickle
import pandas as pd
import numpy as np
from imp import reload
import math
import h5py
import shutil
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, Subset
from itertools import chain
from sklearn.model_selection import KFold
from progiter import ProgIter
from sklearn.model_selection import train_test_split
from mymodule import Trasnformer_model, VAE_model

random_state=34

In [2]:
# all drugs
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcombANDchembl26.pickle','rb') as f:
    a = pickle.load(f)
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb_BY_cid_duplicated.pickle','rb') as f:
    b = pickle.load(f)
smiles = a.append(b).drop_duplicates().reset_index(drop=True)
v = Trasnformer_model.WordVocab(smiles, max_size=None, min_freq=1)
dataset = Trasnformer_model.Seq2seqDataset(smiles, v, seq_len=145)


Building Vocab


In [3]:
PAD = 0 # size of padding
batch_s = 650 # size of molecules per batch
num_w = 6 # num_workers for loading data
best_loss = 10 # initial random high number above ca. 5
n_epoch = 3 # epochs per fold
n_layer = 6 # num_layers of transformer
patience = 5 # patience of scheduler
factor = 0.1 # LR shrink on plateau
hidden = 16 # size of fingerprint
initial_lr=1e-3  # learning rate
n_splits=5 # for CV

model = Trasnformer_model.TrfmSeq2seq(
    batch_size=batch_s,
    in_size=len(v), 
    hidden_size = hidden,
    out_size=len(v),
    n_layers=n_layer
)
m = torch.load('/tf/notebooks/code_for_pub/_logs_as_python_files/transformer_training_logs/16_model_best.pth.tar')
model.load_state_dict(m['state_dict'])
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       factor=factor,
                                                       patience=patience,
                                                       mode='min', 
                                                       min_lr=1e-6,
                                                      verbose=True)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


for e in range(0, n_epoch):
    for ind,(train_index,test_index) in enumerate(kf.split(dataset)):
        train = DataLoader(Subset(dataset, train_index), batch_size=batch_s, shuffle=False, num_workers=num_w)
        test = DataLoader(Subset(dataset, test_index), batch_size=batch_s, shuffle=False, num_workers=num_w)
        
        for b, sm in ProgIter(enumerate(train),total=len(train), verbose=1, show_times=False):
            sm = torch.t(sm.cuda()) # (T,B)
            optimizer.zero_grad()
            output = model(sm) # (T,B,V)

            loss = F.nll_loss(output.view(-1, len(v)), sm.contiguous().view(-1), ignore_index = PAD)
            loss.backward()
            optimizer.step()

            if b%(len(train)//4)==0:
                print(f'\nfold: {ind}, epoch: {e}, iter: {b}, loss {loss:.9f}')
            if b==len(train)-1:
                loss = Trasnformer_model.evaluate(model, test, v, PAD = PAD)
                scheduler.step(loss)
                print(f'\nvalid loss: {loss:.9f}, lr: {optimizer.param_groups[0]["lr"]}')

                # Save the model if the validation loss is the best we've seen so far.
                is_best = loss < best_loss
                if is_best:

                    print(f"currelnt lr: {scheduler.state_dict()['_last_lr'][0]}")
                    print(f"previous best:{best_loss}, now: {loss}")
                    best_loss = loss
                    Trasnformer_model.save_checkpoint({
                        'state_dict' : model.state_dict(), 
                        'best_loss' : best_loss,
                        'vocab' : v,
                        'lr' : scheduler.state_dict()['_last_lr']},
                        is_best, size=hidden)


    0/2210... 
fold: 0, epoch: 0, iter: 0, loss 0.026752619
  552/2210... 
fold: 0, epoch: 0, iter: 552, loss 0.000579581
 1104/2210... 
fold: 0, epoch: 0, iter: 1104, loss 0.000597334
 1656/2210... 
fold: 0, epoch: 0, iter: 1656, loss 0.000254464
 2208/2210... 
fold: 0, epoch: 0, iter: 2208, loss 0.001949811
 2209/2210... 
valid loss: 0.000019487, lr: 0.001
currelnt lr: 0.001
previous best:10, now: 1.9487354258312628e-05
 2210/2210... 
    0/2210... 
fold: 1, epoch: 0, iter: 0, loss 0.000001160
  552/2210... 
fold: 1, epoch: 0, iter: 552, loss 0.000000558
 1104/2210... 
fold: 1, epoch: 0, iter: 1104, loss 0.000000455
 1656/2210... 
fold: 1, epoch: 0, iter: 1656, loss 0.000000409
 2208/2210... 
fold: 1, epoch: 0, iter: 2208, loss 0.000349793
 2209/2210... 
valid loss: 0.000004099, lr: 0.001
currelnt lr: 0.001
previous best:1.9487354258312628e-05, now: 4.09945718366477e-06
 2210/2210... 
    0/2210... 
fold: 2, epoch: 0, iter: 0, loss 0.000000362
  552/2210... 
fold: 2, epoch: 0, iter: 

In [1]:
PAD = 0 # size of padding
batch_s = 280 # size of molecules per batch
num_w = 6 # num_workers for loading data
best_loss = 10 # initial random high number above ca. 5
n_epoch = 3 # epochs per fold
n_layer = 6 # num_layers of transformer
patience = 5 # patience of scheduler
factor = 0.1 # LR shrink on plateau
hidden = 256 # size of fingerprint
initial_lr=1e-4  # learning rate
n_splits=5 # for CV

model = Trasnformer_model.TrfmSeq2seq(
    batch_size=batch_s,
    in_size=len(v), 
    hidden_size = hidden,
    out_size=len(v),
    n_layers=n_layer
)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       factor=factor,
                                                       patience=patience,
                                                       mode='min', 
                                                       min_lr=1e-6,
                                                      verbose=True)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


for e in range(0, n_epoch):
    for ind,(train_index,test_index) in enumerate(kf.split(dataset)):
        train = DataLoader(Subset(dataset, train_index), batch_size=batch_s, shuffle=False, num_workers=num_w)
        test = DataLoader(Subset(dataset, test_index), batch_size=batch_s, shuffle=False, num_workers=num_w)
        
        for b, sm in ProgIter(enumerate(train),total=len(train), verbose=1, show_times=False):
            sm = torch.t(sm.cuda()) # (T,B)
            optimizer.zero_grad()
            output = model(sm) # (T,B,V)

            loss = F.nll_loss(output.view(-1, len(v)), sm.contiguous().view(-1), ignore_index = PAD)
            loss.backward()
            optimizer.step()

            if b%(len(train)//4)==0:
                print(f'\nfold: {ind}, epoch: {e}, iter: {b}, loss {loss:.5f}')
            if b==len(train)-1:
                loss = Trasnformer_model.evaluate(model, test, v, PAD = PAD)
                scheduler.step(loss)
                print(f'\nvalid loss: {loss:.8f}, lr: {optimizer.param_groups[0]["lr"]}')

                # Save the model if the validation loss is the best we've seen so far.
                is_best = loss < best_loss
                if is_best:

                    print(f"currelnt lr: {scheduler.state_dict()['_last_lr'][0]}")
                    print(f"previous best:{best_loss}, now: {loss}")
                    best_loss = loss
                    Trasnformer_model.save_checkpoint({
                        'state_dict' : model.state_dict(), 
                        'best_loss' : best_loss,
                        'vocab' : v,
                        'lr' : scheduler.state_dict()['_last_lr']},
                        is_best, size=hidden)

AssertionError: 